In [1]:
import jax
import jax.numpy as jnp
from jax import lax
import time
import numpy as np
import numba
import functools
from c_tictactoe import check_winner

In [2]:
# Win conditions defined as indices on the board
def check_win1(board: jnp.ndarray) -> int:
    win_conditions = jnp.array(
        [
            (0, 1, 2),
            (3, 4, 5),
            (6, 7, 8),
            (0, 3, 6),
            (1, 4, 8),
            (2, 5, 8),
            (0, 4, 8),
            (2, 4, 6),
        ],
        dtype=jnp.int32,
    )

    def check_line(winner, line):
        line_win = lax.cond(
            (board[line[0]] == board[line[1]])
            & (board[line[1]] == board[line[2]])
            & (board[line[0]] != 0),
            lambda: board[line[0]].astype(jnp.int32),  # Ensuring int32 output
            lambda: jnp.array(0, dtype=jnp.int32),  # Ensuring int32 output
        )
        return jnp.maximum(winner, line_win), None

    # Use `jnp.array(0)` as the initial carry value, which represents "no winner"
    winner, _ = lax.scan(check_line, jnp.array(0), win_conditions)
    return winner  # Returns 1 if player wins, 2 if opponent wins, 0 otherwise

In [3]:
def check_win2(board: jnp.ndarray) -> int:
    reshaped_state = board.reshape(3, 3)
    # 0 for tie, 1 for player 1, 2 for player 2
    # Check for row-wise winners
    rows_equal = jnp.all(reshaped_state == reshaped_state[:, [0]], axis=1) & (
        reshaped_state[:, 0] != 0
    )
    row_winners = reshaped_state[:, 0] * rows_equal

    # Check for column-wise winners
    cols_equal = jnp.all(reshaped_state == reshaped_state[[0], :], axis=0) & (
        reshaped_state[0, :] != 0
    )
    col_winners = reshaped_state[0, :] * cols_equal

    # Check for diagonal winners
    diagonal_1 = (
        (reshaped_state[0, 0] != 0)
        & (reshaped_state[0, 0] == reshaped_state[1, 1])
        & (reshaped_state[0, 0] == reshaped_state[2, 2])
    )
    diagonal_1_winners = reshaped_state[0, 0] * diagonal_1

    diagonal_2 = (
        (reshaped_state[0, 2] != 0)
        & (reshaped_state[0, 2] == reshaped_state[1, 1])
        & (reshaped_state[0, 2] == reshaped_state[2, 0])
    )
    diagonal_2_winners = reshaped_state[0, 2] * diagonal_2

    # Return the maximum winner (1 for player 1, 2 for player 2, 0 for no winner)
    return jnp.maximum(
        row_winners.max(),
        jnp.maximum(
            col_winners.max(),
            jnp.maximum(diagonal_1_winners, diagonal_2_winners),
        ),
    )

In [4]:
def check_win2_np(board: np.ndarray) -> int:
    reshaped_state = board.reshape(3, 3)
    # 0 for tie, 1 for player 1, 2 for player 2
    # Check for row-wise winners
    rows_equal = np.all(reshaped_state == reshaped_state[:, [0]], axis=1) & (
        reshaped_state[:, 0] != 0
    )
    row_winners = reshaped_state[:, 0] * rows_equal

    # Check for column-wise winners
    cols_equal = np.all(reshaped_state == reshaped_state[[0], :], axis=0) & (
        reshaped_state[0, :] != 0
    )
    col_winners = reshaped_state[0, :] * cols_equal

    # Check for diagonal winners
    diagonal_1 = (
        (reshaped_state[0, 0] != 0)
        & (reshaped_state[0, 0] == reshaped_state[1, 1])
        & (reshaped_state[0, 0] == reshaped_state[2, 2])
    )
    diagonal_1_winners = reshaped_state[0, 0] * diagonal_1

    diagonal_2 = (
        (reshaped_state[0, 2] != 0)
        & (reshaped_state[0, 2] == reshaped_state[1, 1])
        & (reshaped_state[0, 2] == reshaped_state[2, 0])
    )
    diagonal_2_winners = reshaped_state[0, 2] * diagonal_2

    # Return the maximum winner (1 for player 1, 2 for player 2, 0 for no winner)
    return max(
        row_winners.max(),
        max(
            col_winners.max(),
            max(diagonal_1_winners, diagonal_2_winners),
        ),
    )

In [5]:
def check_win3(board: jnp.ndarray) -> int:
    # 0 for tie, 1 for player 1, 2 for player 2
    reshaped_state = board.reshape(3, 3)
    # rows
    rows_equal = jnp.all(reshaped_state == reshaped_state[:, [0]], axis=1) & (
        reshaped_state[:, 0] != 0
    )
    if jnp.any(rows_equal):
        return reshaped_state[rows_equal][0, 0]
    # columns
    cols_equal = jnp.all(reshaped_state == reshaped_state[[0], :], axis=0) & (
        reshaped_state[0, :] != 0
    )
    if jnp.any(cols_equal):
        return reshaped_state[0, cols_equal][0]
    # diagonals
    if reshaped_state[0, 0] == reshaped_state[1, 1] == reshaped_state[2, 2] != 0:
        return reshaped_state[0, 0]
    if reshaped_state[2, 0] == reshaped_state[1, 1] == reshaped_state[0, 2] != 0:
        return reshaped_state[2, 0]
    return 0

In [6]:
def check_win3_np(board: np.ndarray) -> int:
    # 0 for tie, 1 for player 1, 2 for player 2
    reshaped_state = board.reshape(3, 3)
    # rows
    rows_equal = np.all(reshaped_state == reshaped_state[:, [0]], axis=1) & (
        reshaped_state[:, 0] != 0
    )
    if np.any(rows_equal):
        return reshaped_state[rows_equal][0, 0]
    # columns
    cols_equal = np.all(reshaped_state == reshaped_state[[0], :], axis=0) & (
        reshaped_state[0, :] != 0
    )
    if np.any(cols_equal):
        return reshaped_state[0, cols_equal][0]
    # diagonals
    if reshaped_state[0, 0] == reshaped_state[1, 1] == reshaped_state[2, 2] != 0:
        return reshaped_state[0, 0]
    if reshaped_state[2, 0] == reshaped_state[1, 1] == reshaped_state[0, 2] != 0:
        return reshaped_state[2, 0]
    return 0

In [7]:
@numba.njit
def check_win3_numba(board: np.ndarray) -> int:
    # Reshape the board
    reshaped_state = board.reshape(3, 3)
    
    # Check rows
    for row in range(3):
        if reshaped_state[row, 0] != 0 and reshaped_state[row, 0] == reshaped_state[row, 1] == reshaped_state[row, 2]:
            return reshaped_state[row, 0]
    
    # Check columns
    for col in range(3):
        if reshaped_state[0, col] != 0 and reshaped_state[0, col] == reshaped_state[1, col] == reshaped_state[2, col]:
            return reshaped_state[0, col]
    
    # Check diagonals
    if reshaped_state[0, 0] != 0 and reshaped_state[0, 0] == reshaped_state[1, 1] == reshaped_state[2, 2]:
        return reshaped_state[0, 0]
    if reshaped_state[2, 0] != 0 and reshaped_state[2, 0] == reshaped_state[1, 1] == reshaped_state[0, 2]:
        return reshaped_state[2, 0]
    
    # No winner
    return 0

In [8]:
def check_win4(board: jnp.ndarray) -> int:
    win_conditions = jnp.array(
        [
            [0, 1, 2],  # Rows
            [3, 4, 5],
            [6, 7, 8],
            [0, 3, 6],  # Columns
            [1, 4, 7],
            [2, 5, 8],
            [0, 4, 8],  # Diagonals
            [2, 4, 6],
        ],
        dtype=jnp.int32,
    )
    # Extract the values at the win conditions
    lines = board[win_conditions]  # Shape: (8, 3)

    # Check if all elements in a line are the same and not zero
    lines_equal = (lines[:, 0] == lines[:, 1]) & (lines[:, 1] == lines[:, 2]) & (lines[:, 0] != 0)

    # Get the winner for each line
    line_winners = lines[:, 0] * lines_equal  # Winner is lines[:, 0] if line is equal, else zero

    # Return the maximum winner (1 or 2 if there's a winner, 0 otherwise)
    return jnp.max(line_winners)

In [9]:
def check_win5(board: jnp.ndarray) -> int:
    win_conditions = jnp.array(
        [
            [0, 1, 2],  # Rows
            [3, 4, 5],
            [6, 7, 8],
            [0, 3, 6],  # Columns
            [1, 4, 7],
            [2, 5, 8],
            [0, 4, 8],  # Diagonals
            [2, 4, 6],
        ],
        dtype=jnp.int32,
    )

    # Gather the board values at the win condition indices
    lines = board[win_conditions]  # Shape: (8, 3)

    # Check if all elements in a line are equal and not zero
    lines_equal = (lines == lines[:, [0]]) & (lines[:, 0:1] != 0)
    winners = lines[:, 0] * jnp.all(lines_equal, axis=1)

    # Return the maximum winner (1 or 2 if there's a winner, 0 otherwise)
    return jnp.max(winners)

In [10]:
def benchmark(func):
    key, rng = jax.random.split(jax.random.PRNGKey(0), 2)
    for _ in range(5):
        key, rng = jax.random.split(key, 2)
        board= jax.random.randint(rng, (9,), 0, 3)
        func(board)

def benchmark_np(func):
    for _ in range(5):
        board = np.random.randint(0, 3, 9)
        func(board)

@functools.partial(jax.jit, static_argnums=(0,))
def benchmark_jax(func):
    key, rng = jax.random.split(jax.random.PRNGKey(0), 2)
    for _ in range(5):
        key, rng = jax.random.split(key, 2)
        board= jax.random.randint(rng, (9,), 0, 3)
        jax.block_until_ready(func(board))

In [11]:
jit_check_win1 = jax.jit(check_win1)
jit_check_win2 = jax.jit(check_win2)
jit_check_win3 = jax.jit(check_win3)
jit_check_win4 = jax.jit(check_win4)
jit_check_win5 = jax.jit(check_win5)
jit_check_win1(jnp.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))
jit_check_win2(jnp.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))
# jit_check_win3(jnp.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))
jit_check_win4(jnp.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))
jit_check_win5(jnp.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))

Array(1, dtype=int32)

In [12]:
%timeit benchmark(check_win1)
%timeit benchmark(jit_check_win1)
%timeit benchmark(check_win2)
%timeit benchmark(jit_check_win2)
%timeit benchmark(check_win3)
%timeit benchmark(check_win4)
%timeit benchmark(jit_check_win4)
%timeit benchmark(check_win5)
%timeit benchmark(jit_check_win5)

269 ms ± 23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


KeyboardInterrupt: 

In [ ]:
%timeit benchmark_jax(check_win1)
%timeit benchmark_jax(jit_check_win1)
%timeit benchmark_jax(check_win2)
%timeit benchmark_jax(jit_check_win2)
%timeit benchmark_jax(check_win4)
%timeit benchmark_jax(jit_check_win4)
%timeit benchmark_jax(check_win5)
%timeit benchmark_jax(jit_check_win5)

2.41 μs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.45 μs ± 93.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.63 μs ± 197 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.82 μs ± 354 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.49 μs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.43 μs ± 70.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.42 μs ± 51.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.53 μs ± 137 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
%timeit benchmark_np(check_win2_np)
%timeit benchmark_np(check_win3_np)

180 μs ± 6.81 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
157 μs ± 1.9 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
check_win3_numba(np.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))

1

In [ ]:
%timeit benchmark_np(check_win3_numba)

35.2 μs ± 8.77 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit benchmark_np(check_winner)

32.4 μs ± 645 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
jax.make_jaxpr(check_win1)(np.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))

{ lambda a:i32[8,3]; b:i32[9]. let
    c:i32[] = scan[
      _split_transpose=False
      jaxpr={ lambda ; d:i32[9] e:i32[] f:i32[3]. let
          g:i32[1] = slice[limit_indices=(1,) start_indices=(0,) strides=None] f
          h:i32[] = squeeze[dimensions=(0,)] g
          i:bool[] = lt h 0
          j:i32[] = add h 9
          k:i32[] = select_n i h j
          l:i32[1] = dynamic_slice[slice_sizes=(1,)] d k
          m:i32[] = squeeze[dimensions=(0,)] l
          n:i32[1] = slice[limit_indices=(2,) start_indices=(1,) strides=None] f
          o:i32[] = squeeze[dimensions=(0,)] n
          p:bool[] = lt o 0
          q:i32[] = add o 9
          r:i32[] = select_n p o q
          s:i32[1] = dynamic_slice[slice_sizes=(1,)] d r
          t:i32[] = squeeze[dimensions=(0,)] s
          u:bool[] = eq m t
          v:i32[1] = slice[limit_indices=(2,) start_indices=(1,) strides=None] f
          w:i32[] = squeeze[dimensions=(0,)] v
          x:bool[] = lt w 0
          y:i32[] = add w 9
    

In [ ]:
jax.make_jaxpr(check_win2)(np.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))

{ lambda a:i32[1] b:i32[1]; c:i32[9]. let
    d:i32[3,3] = reshape[dimensions=None new_sizes=(3, 3)] c
    e:bool[1] = lt a 0
    f:i32[1] = add a 3
    g:i32[1] = select_n e a f
    h:i32[1,1] = broadcast_in_dim[broadcast_dimensions=(0,) shape=(1, 1)] g
    i:i32[3,1] = gather[
      dimension_numbers=GatherDimensionNumbers(offset_dims=(0,), collapsed_slice_dims=(1,), start_index_map=(1,), operand_batching_dims=(), start_indices_batching_dims=())
      fill_value=None
      indices_are_sorted=False
      mode=GatherScatterMode.PROMISE_IN_BOUNDS
      slice_sizes=(3, 1)
      unique_indices=False
    ] d h
    j:bool[3,3] = eq d i
    k:bool[3] = reduce_and[axes=(1,)] j
    l:i32[3,1] = slice[limit_indices=(3, 1) start_indices=(0, 0) strides=None] d
    m:i32[3] = squeeze[dimensions=(1,)] l
    n:bool[3] = ne m 0
    o:bool[3] = and k n
    p:i32[3,1] = slice[limit_indices=(3, 1) start_indices=(0, 0) strides=None] d
    q:i32[3] = squeeze[dimensions=(1,)] p
    r:i32[3] = convert_eleme

In [ ]:
jax.make_jaxpr(check_win4)(np.array([1, 1, 1, 0, 0, 0, 0, 0, 0]))

{ lambda a:i32[8,3]; b:i32[9]. let
    c:bool[8,3] = lt a 0
    d:i32[8,3] = add a 9
    e:i32[8,3] = select_n c a d
    f:i32[8,3,1] = broadcast_in_dim[broadcast_dimensions=(0, 1) shape=(8, 3, 1)] e
    g:i32[8,3] = gather[
      dimension_numbers=GatherDimensionNumbers(offset_dims=(), collapsed_slice_dims=(0,), start_index_map=(0,), operand_batching_dims=(), start_indices_batching_dims=())
      fill_value=None
      indices_are_sorted=False
      mode=GatherScatterMode.PROMISE_IN_BOUNDS
      slice_sizes=(1,)
      unique_indices=False
    ] b f
    h:i32[8,1] = slice[limit_indices=(8, 1) start_indices=(0, 0) strides=None] g
    i:i32[8] = squeeze[dimensions=(1,)] h
    j:i32[8,1] = slice[limit_indices=(8, 2) start_indices=(0, 1) strides=None] g
    k:i32[8] = squeeze[dimensions=(1,)] j
    l:bool[8] = eq i k
    m:i32[8,1] = slice[limit_indices=(8, 2) start_indices=(0, 1) strides=None] g
    n:i32[8] = squeeze[dimensions=(1,)] m
    o:i32[8,1] = slice[limit_indices=(8, 3) start_ind

In [ ]:
def benchmark_empty():
    key, rng = jax.random.split(jax.random.PRNGKey(0), 2)
    for _ in range(5):
        key, rng = jax.random.split(key, 2)
        board=jax.random.randint(rng, (9,), 0, 3)

def benchmark_np_empty():
    for _ in range(5):
        board = np.random.randint(0, 3, 9)

@functools.partial(jax.jit, static_argnums=(0,))
def benchmark_jax_emtpy(func):
    key, rng = jax.random.split(jax.random.PRNGKey(0), 2)
    for _ in range(5):
        key, rng = jax.random.split(key, 2)
        board= jax.random.randint(rng, (9,), 0, 3)

@numba.njit
def benchmark_numba_empty():
    for _ in range(5):
        board = np.random.randint(0, 3, 9)

In [ ]:
%timeit benchmark_empty()
%timeit benchmark_np_empty()
%timeit benchmark_jax_emtpy(benchmark_empty)
%timeit benchmark_numba_empty()

730 μs ± 17.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
27.4 μs ± 613 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
2.44 μs ± 110 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
The slowest run took 9.56 times longer than the fastest. This could mean that an intermediate result is being cached.
2.59 μs ± 2.51 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# jax.profiler.start_server(port=6007)
# with jax.profiler.trace('jax_trace', create_perfetto_link=True):
#     key, rng = jax.random.split(jax.random.PRNGKey(0), 2)
#     for _ in range(1000):
#         key, rng = jax.random.split(key, 2)
#         board= jax.random.randint(rng, (9,), 0, 3)
#         jit_check_win4(board)